Environment Setup Module.
This cell installs the necessary quantum computing libraries in the Google Colab environment.
It must be executed before running any other cell.

In [ ]:
!pip install qiskit[visualization] qiskit-aer matplotlib pylatexenc

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

print("\n ENVIRONMENT SETUP COMPLETE! You can now proceed to the next cell.")

### Exercise 4: The Reality of Measurement (Shot Noise)

**Motivation:**
In classical computing, reading a bit is a deterministic process: if a bit is `1`, it will always be read as `1`. In quantum mechanics, measuring a state in superposition (like the $|+\rangle$ state created by the Hadamard gate) forces the wavefunction to collapse probabilistically. 

**Theoretical Context:**
According to the theory you studied, the H gate creates the state:
$$|+\rangle = \frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle$$
The probability of measuring `0` or `1` is exactly 50% ($|1/\sqrt{2}|^2 = 0.5$). However, this is a *theoretical* probability. In the real world, measuring a quantum state is like flipping a coin.

**Implications & Your Task:**
If you flip a fair coin 10 times, you rarely get exactly 5 Heads and 5 Tails. This statistical variance is called **Shot Noise**. To extract reliable information from a quantum computer, we must execute the same circuit many times (called "shots").
1. Run the cell below with `NUM_SHOTS = 50`. Observe the deviation from the ideal 50/50 split.
2. Change `NUM_SHOTS` to `100`, then `1000`, then `10000`. 
3. **Observe:** See how the histogram gradually converges to the theoretical 50% distribution. This proves why real quantum algorithms require thousands of repeated executions.

In [ ]:
"""
Shot Noise Experiment Module.
Demonstrates the probabilistic nature of measuring a quantum superposition 
and the effect of the law of large numbers on measurement statistics.
"""

# --- STUDENT EXPERIMENT ZONE ---
NUM_SHOTS = 50  # Try changing this to 100, 1000, or 10000
# -------------------------------

def run_superposition_experiment(shots: int) -> dict:
    """
    Creates a superposition state, measures it, and returns the results.
    
    Args:
        shots (int): The number of times the quantum circuit is executed.
        
    Returns:
        dict: A dictionary containing the measurement counts (e.g., {'0': 22, '1': 28}).
    """
    # Initialize a quantum circuit with 1 qubit and 1 classical bit
    qc = QuantumCircuit(1, 1)
    
    # Apply Hadamard gate to create a 50/50 superposition
    qc.h(0)            
    
    # Measure the qubit and store the result in the classical bit
    qc.measure(0, 0)   
    
    # Execute the circuit on the simulator
    simulator = AerSimulator()
    job = simulator.run(qc, shots=shots)
    result = job.result()
    
    return result.get_counts()

# Execute the experiment
counts = run_superposition_experiment(NUM_SHOTS)

# Display results
print(f"Results after {NUM_SHOTS} shots: {counts}")
expected = NUM_SHOTS / 2
diff = abs(counts.get('0', 0) - expected)
print(f"Deviation from ideal 50/50 split: {diff} shots")

# Plot the histogram
plot_histogram(counts)

### Exercise 5: Quantum Interference (The H-Z-H Sequence)

**Motivation:**
In the Composer exercise, you applied a Z gate to a superposition. The visual result was a color change on the Q-Sphere (from Blue to Red, indicating a negative phase), but the probability bar chart did not change at all. It remained 50/50. 
*How can we prove that the Z gate actually did something if the measurement doesn't change?*

**Theoretical Context:**
Quantum mechanics allows us to use **Interference** to convert "invisible" phase differences into "visible" probability changes. 
By applying a second Hadamard gate, we mix the amplitudes again. 
* If the phase is positive (No Z gate), the amplitudes for $|1\rangle$ cancel out (**Destructive Interference**), leaving 100% $|0\rangle$.
* If the phase is negative (With Z gate), the amplitudes for $|0\rangle$ cancel out, leaving 100% $|1\rangle$.

**Implications & Your Task:**
This ability to manipulate unseen phases to amplify correct answers and cancel out wrong answers is the core engine of powerful algorithms like Shor's and Grover's.
1. Run the cell below *without* the Z gate. Notice the output is deterministically `0`.
2. Remove the `#` symbol in front of `qc_int.z(0)` to activate the phase flip.
3. Run the cell again. **Observe:** The output completely flips to `1`. You have just used interference to detect a hidden quantum phase!

In [ ]:
"""
Quantum Interference Module.
Demonstrates how phase differences (created by the Z gate) can be converted 
into measurable amplitude differences using quantum interference (H gates).
"""

def run_interference_experiment() -> None:
    """
    Constructs and executes the H-Z-H interference circuit, then plots the results.
    """
    qc_int = QuantumCircuit(1, 1)

    # Step 1: Create initial superposition (|+> state)
    qc_int.h(0)

    # Step 2: Phase Flip
    # --- STUDENT EXPERIMENT ZONE ---
    # Remove the '#' from the line below to activate the Z gate
    
    # qc_int.z(0)
    
    # -------------------------------

    # Step 3: Close the interference loop
    qc_int.h(0)

    # Measure the final state
    qc_int.measure(0, 0)
    
    # Draw the circuit for visual confirmation
    display(qc_int.draw('mpl'))

    # Execute and plot
    simulator = AerSimulator()
    job = simulator.run(qc_int, shots=1000)
    counts = job.result().get_counts()
    
    print(f"Final Measurement: {counts}")
    display(plot_histogram(counts))

# Execute the experiment
run_interference_experiment()